In [1]:
import os
import pickle as pkl
import networkx as nx
import convokit
from convokit import Corpus, Utterance, Speaker, HyperConvo, VectorClassifier, BoWTransformer, download
from convokit.prompt_types import PromptTypeWrapper
from sklearn.model_selection import train_test_split

In [6]:
data = pkl.load(open('./serializedGraphs/444.pkl', 'rb'))

In [7]:
data.node

{'1470.1240': {'edited': 1492393763004,
  'votes': [0, 0, 0, 0, 0],
  'relation': 1,
  'created': 1492393763004,
  'text': 'Politicians, in contrast to millions of voters from different walks of life, are more prone to mistakes due to their small numbers and their relative sameness in many regards.',
  'author': '9972cb14-00f7-42d2-9b64-846aa0812775'},
 '1470.1244': {'edited': 1492505771591,
  'votes': [0, 0, 0, 0, 0],
  'created': 1492505771591,
  'text': 'Politicians work within political systems with checks and balances that limit their power and influence.',
  'author': '24f5ff14-75af-4548-ae24-9751d29975f4',
  'relation': 1},
 '1470.1245': {'edited': 1492506361173,
  'votes': [0, 0, 0, 0, 0],
  'relation': 1,
  'created': 1492506361173,
  'text': 'As elections take place at regular intervals, citizens have the opportunity to vote bad politicians out of office. Compared to that, bad subject matter decisions are usually long-term oriented, and sometimes irreversible.',
  'author': '

In [8]:
data.edge

{'1470.1240': {'1470.1122': {'weight': 1}},
 '1470.1244': {'1470.1086': {'weight': 1}},
 '1470.1245': {'1470.1086': {'weight': 1}},
 '1470.1246': {'1470.1086': {'weight': 1}},
 '1470.1248': {'1470.1086': {'weight': -1}},
 '1470.1249': {'1470.1246': {'weight': 1}},
 '1470.817': {'1470.144': {'weight': -1}},
 '1470.816': {'1470.763': {'weight': -1}},
 '1470.815': {'1470.763': {'weight': -1}},
 '1470.814': {'1470.813': {'weight': 1}},
 '1470.813': {'1470.85': {'weight': -1}},
 '1470.473': {'1470.294': {'weight': -1}},
 '1470.956': {'1470.947': {'weight': 1}},
 '1470.957': {'1470.25': {'weight': 1}},
 '1470.954': {'1470.7': {'weight': -1}},
 '1470.472': {'1470.306': {'weight': -1}},
 '1470.596': {'1470.560': {'weight': 1}},
 '1470.594': {'1470.549': {'weight': 1}},
 '1470.595': {'1470.594': {'weight': 1}},
 '1470.619': {'1470.549': {'weight': 1}},
 '1470.618': {'1470.549': {'weight': 1}},
 '1470.590': {'1470.555': {'weight': 1}},
 '1470.591': {'1470.586': {'weight': -1}},
 '1470.1395': {'1

In [10]:
#### Rough work
t = []
for node_id in data.node:
    t.append(int(node_id.split('.')[1]))
t.sort()
print(len(t))
print(t)

925
[0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 29, 30, 31, 32, 33, 36, 37, 38, 39, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 59, 60, 61, 62, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 122, 123, 124, 125, 126, 127, 128, 129, 130, 135, 136, 137, 138, 139, 140, 143, 144, 145, 146, 147, 148, 153, 154, 155, 156, 157, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 179, 193, 199, 200, 201, 202, 203, 204, 205, 206, 207, 214, 215, 216, 217, 218, 219, 225, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 245, 246, 252, 253, 254, 255, 259, 263, 264, 265, 266, 269, 270, 271, 272, 275, 276, 277, 281, 282, 283, 284, 287, 288, 294, 295, 298, 299, 300, 301, 303, 304, 305, 306, 307, 308, 309, 310, 311, 316, 317, 318, 319, 320, 324, 325, 330, 331, 332, 333, 335, 

In [3]:
files = os.listdir('./serializedGraphs/')

In [3]:
def create_utterance(node_id, node, edge):
    conv_id = node_id.split('.')[0] + '.0'
    parent = None
    speaker = Speaker(id=node['author'])
    
    if len(edge.keys()) >= 1:
        parent = list(edge.keys())[0]
        
    return Utterance(id=node_id, speaker=speaker, conversation_id=conv_id, reply_to=parent,
                     timestamp=node['created'], text=node['text'], meta={'label': node['relation']})

In [4]:
utterances_list = []

for file in files:
    data = pkl.load(open('./serializedGraphs/' + file, 'rb'))
    for node_id in data.node.keys():
        utterances_list.append(create_utterance(node_id, data.node[node_id], data.edge[node_id]))

In [5]:
len(utterances_list)

329013

In [6]:
corpus = Corpus(utterances=utterances_list)

In [7]:
corpus.print_summary_stats()

Number of Speakers: 21326
Number of Utterances: 329013
Number of Conversations: 1560


## Compute Q-value

In [4]:
q_values = []
        
for file in files:
    data = pkl.load(open('./serializedGraphs/' + file, 'rb'))
    supporters = 0
    attackers = 0
    for node_id in data.node.keys():
        if data.node[node_id]['relation'] == -1:
            attackers += 1
        elif data.node[node_id]['relation'] == 1:
            supporters += 1
    q_ratio= supporters / (supporters + attackers)
    q_values.append(q_ratio)

In [7]:
buckets = [0, 0, 0, 0, 0]
for val in q_values:
    if val <= 0.2:
        buckets[0] += 1
    elif 0.2 < val <= 0.4:
        buckets[1] += 1
    elif 0.4 < val <= 0.6:
        buckets[2] += 1
    elif 0.6 < val <= 0.8:
        buckets[3] += 1
    elif 0.8 > val:
        buckets[4] += 1

buckets

[11, 455, 795, 279, 0]

## Bag of Words features

In [16]:
utterances_list = list(corpus.iter_utterances())

In [17]:
train_utt, test_utt = train_test_split(utterances_list, test_size=0.2)

In [18]:
print(len(train_utt), len(test_utt))

263210 65803


In [19]:
for utt in train_utt:
    utt.meta['train_test_type'] = 'train'
    
for utt in test_utt:
    utt.meta['train_test_type'] = 'test'

In [8]:
# hc = HyperConvo(prefix_len=10, min_convo_len=10, invalid_val=-1)
# hc.fit_transform(corpus)

bow_transformer = BoWTransformer(obj_type="utterance", vector_name='bow_vector')
bow_transformer.fit_transform(corpus)

Initializing default unigram CountVectorizer...Done.


In [9]:
corpus.vectors

{'bow_vector'}

In [ ]:
corpus.get_vectors('bow_vector', as_dataframe=True).shape

In [22]:
hc_classifier = VectorClassifier(obj_type="utterance", vector_name='bow_vector',
                                   labeller=lambda utt: utt.meta['label'])

Initialized default classification model (standard scaled logistic regression).


In [23]:
hc_classifier.fit(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'train')

/usr/local/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [24]:
hc_classifier.transform(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'test')

In [25]:
print(hc_classifier.classification_report(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'test'))

              precision    recall  f1-score   support

          -1       0.70      0.75      0.72     36881
           0       0.53      0.55      0.54       933
           1       0.63      0.56      0.60     27989

    accuracy                           0.67     65803
   macro avg       0.62      0.62      0.62     65803
weighted avg       0.67      0.67      0.67     65803



## Pragmatic features: PromptTypes (Rhetorics)

In [58]:
#DATA_DIR = './'
#FULL_ROOT_DIR = download('wiki-corpus', data_dir=DATA_DIR)

FULL_ROOT_DIR = './wiki-corpus'

full_corpus = Corpus(FULL_ROOT_DIR)
full_corpus.load_info('utterance',['parsed'])

/usr/local/lib/python3.8/site-packages/convokit/model/corpus.py:1054: FutureWarning: set_info() is deprecated and will be removed in a future release. Use add_meta() instead.


In [60]:
pt_model = PromptTypeWrapper(n_types=6, use_prompt_motifs=True, root_only=True,
                            questions_only=False, enforce_caps=False, min_support=20, min_df=100,
                            svd__n_components=50, max_dist=1., random_state=1000)
pt_model.fit(full_corpus)

/usr/local/lib/python3.8/site-packages/convokit/prompt_types/promptTypeWrapper.py:52: FutureWarning: get_info() is deprecated and will be removed in a future release. Use retrieve_meta() instead.


10000/391294 utterances processed
20000/391294 utterances processed
30000/391294 utterances processed
40000/391294 utterances processed
50000/391294 utterances processed
60000/391294 utterances processed
70000/391294 utterances processed
80000/391294 utterances processed
90000/391294 utterances processed
100000/391294 utterances processed
110000/391294 utterances processed
120000/391294 utterances processed
130000/391294 utterances processed
140000/391294 utterances processed
150000/391294 utterances processed
160000/391294 utterances processed
170000/391294 utterances processed
180000/391294 utterances processed
190000/391294 utterances processed
200000/391294 utterances processed
210000/391294 utterances processed
220000/391294 utterances processed
230000/391294 utterances processed
240000/391294 utterances processed
250000/391294 utterances processed
260000/391294 utterances processed
270000/391294 utterances processed
280000/391294 utterances processed
290000/391294 utterances proc

/usr/local/lib/python3.8/site-packages/convokit/prompt_types/promptTypeWrapper.py:54: FutureWarning: get_info() is deprecated and will be removed in a future release. Use retrieve_meta() instead.


10000/391294 utterances processed
20000/391294 utterances processed
30000/391294 utterances processed
40000/391294 utterances processed
50000/391294 utterances processed
60000/391294 utterances processed
70000/391294 utterances processed
80000/391294 utterances processed
90000/391294 utterances processed
100000/391294 utterances processed
110000/391294 utterances processed
120000/391294 utterances processed
130000/391294 utterances processed
140000/391294 utterances processed
150000/391294 utterances processed
160000/391294 utterances processed
170000/391294 utterances processed
180000/391294 utterances processed
190000/391294 utterances processed
200000/391294 utterances processed
210000/391294 utterances processed
220000/391294 utterances processed
230000/391294 utterances processed
240000/391294 utterances processed
250000/391294 utterances processed
260000/391294 utterances processed
270000/391294 utterances processed
280000/391294 utterances processed
290000/391294 utterances proc

KeyboardInterrupt: 

In [ ]:
pt_model.transform(corpus)
corpus.vectors

In [ ]:
pt_classifier = VectorClassifier(obj_type="utterance", vector_name='bow_vector',
                                   labeller=lambda utt: utt.meta['label'])

In [ ]:
pt_classifier.fit(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'train')

In [ ]:
pt_classifier.transform(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'test')

In [ ]:
print(pt_classifier.classification_report(corpus, selector=lambda utt: utt.meta['train_test_type'] == 'test'))